In [1]:
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document


/Users/sumanyadav/Desktop/Learn/Learn-RAG/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Step 1 - Sample Documnets
docs = [
    Document(page_content="LangChain is a framework for building applications with LLMs."),
    Document(page_content="Pinecone is a vector database for sematic search"),
    Document(page_content="The Eiffel Tower is located in Paris."),
    Document(page_content="Langchain can be used to develop agentic ai application."),
    Document(page_content="langchain has many types of retrievers")
]

# step 2 - Dense Retrieve (FAISS + HuggingFace)

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
dense_vectorstore = FAISS.from_documents(docs, embedding_model)
dense_retriever = dense_vectorstore.as_retriever()

In [3]:
## Sparse Retriever
sparse_retriever = BM25Retriever.from_documents(docs)
sparse_retriever.k = 3


# step 4 - Combine with Ensamble Retriever
hybrid_retriever = EnsembleRetriever(
    retrievers=[dense_retriever, sparse_retriever],
    weight=[0.7,0.3]
)

In [4]:
hybrid_retriever

EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x176dac1a0>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x176dac830>, k=3)], weights=[0.5, 0.5])

In [5]:
# step 5 - Query and get result
query = "How can i build an application using LLMs?"

results = hybrid_retriever.invoke(query)
print(results)

for i, doc in enumerate(results):
    print(f"\nDoument {i+1}: {doc.page_content}")


[Document(id='06145013-8e42-42ad-990f-68394b8acb5f', metadata={}, page_content='Langchain can be used to develop agentic ai application.'), Document(id='1d594fc6-5c79-4f21-90dd-c1f7d897634e', metadata={}, page_content='langchain has many types of retrievers'), Document(id='555776f0-d971-4dbb-90ec-fff0ff8547ac', metadata={}, page_content='LangChain is a framework for building applications with LLMs.'), Document(id='a719b496-e1b8-460f-8b5a-d8be3de52153', metadata={}, page_content='Pinecone is a vector database for sematic search'), Document(metadata={}, page_content='The Eiffel Tower is located in Paris.')]

Doument 1: Langchain can be used to develop agentic ai application.

Doument 2: langchain has many types of retrievers

Doument 3: LangChain is a framework for building applications with LLMs.

Doument 4: Pinecone is a vector database for sematic search

Doument 5: The Eiffel Tower is located in Paris.


# RAG PIPELINE WITH HYBRID

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain

In [21]:
# Step 6 - prompt template

prompt = PromptTemplate.from_template("""
Answer the question based on the following context:
context: {context}
Question: {input}
""")
prompt

PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the question based on the following context:\ncontext: {context}\nQuestion: {input}\n')

In [30]:
# step 7 - LLMs
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.4
)
llm

ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x14b38ab10>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x14b38aea0>, root_client=<openai.OpenAI object at 0x14b38a780>, root_async_client=<openai.AsyncOpenAI object at 0x14b38ac40>, temperature=0.4, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [31]:
# step 8 - Ragchain
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

rag_chain = create_retrieval_chain(retriever=hybrid_retriever, combine_docs_chain=document_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x176dac1a0>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x176dac830>, k=3)], weights=[0.5, 0.5]), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the question based on the following context:\ncontext: {context}\nQuestion: {input}\n')
            | ChatOpenAI(profile={'max_input_to

In [32]:
query = {"input": "How can I build an application using LLMs?"}

response = rag_chain.invoke(query)
print(f"Answer: {response['answer']}")

for i, doc in enumerate(response['context']):
    print(f"\nDocument {i+1}: {doc.page_content}")

Answer: You can build an application using LLMs by using the LangChain framework, which is specifically designed for developing applications with Large Language Models (LLMs). LangChain provides various types of retrievers that can be used to interact with LLMs and create agentic AI applications.

Document 1: Langchain can be used to develop agentic ai application.

Document 2: langchain has many types of retrievers

Document 3: LangChain is a framework for building applications with LLMs.

Document 4: Pinecone is a vector database for sematic search

Document 5: The Eiffel Tower is located in Paris.
